## **Semantic Search System Using Document Embeddings and Vector Databases**

#### **Introduction:**
In today’s world, the ability to efficiently search and retrieve relevant information from large document collections is crucial for businesses and organizations. Traditional keyword-based search systems often fall short when dealing with complex queries or when users are unsure of the exact terms to search for. This is where **semantic search** comes into play.

Semantic search leverages **natural language processing (NLP)** and **machine learning** to understand the meaning behind words and phrases, enabling more accurate and context-aware retrieval of information. In this lecture, we will explore how to build a semantic search system using **document embeddings** and **vector databases**.

#### **Key Concepts Covered:**
1. **Document Embeddings**: Representing text as high-dimensional vectors that capture semantic meaning.
2. **Vector Databases**: Storing and querying embeddings efficiently for fast retrieval.
3. **Visualizing Embeddings**: Using dimensionality reduction techniques like t-SNE to visualize high-dimensional data in 2D.


In [ ]:
# importing libraries
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


import os
import glob
from dotenv import load_dotenv
import gradio as gr

#### **Configuring the Model and Database**


In [ ]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

#### **Loading Environment Variables**


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")
os.environ['OPENAI_API_KEY'] = openai_api_key

#### **Loading and Preprocessing Documents**

let's load the documents from the subfolders in the `Palazon Global Database` directory.

In [ ]:
folders = glob.glob("../Palazon Global Database/*")
text_loader_kwargs = {'encoding': 'utf-8'}
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

#### **Splitting Documents into Chunks**


let's split the documents into smaller chunks (1000 characters each) with an overlap of 200 characters to ensure context is preserved across chunks.

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

#### **Analyzing Document Types**

Let's identify and print the unique document types found in the dataset, to provide insights into the data structure.

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

#### **Generating Embeddings using OpenAI**

OpenAI embeddings are generated for each text chunk, converting text into high-dimensional vectors that capture semantic meaning.

In [ ]:
embeddings = OpenAIEmbeddings()

### **Next step: Indexing**  

Indexing in a **RAG (Retrieval-Augmented Generation) system** involves converting documents into **vector embeddings** and storing them in a **vector database** for efficient retrieval. Here’s what actually happens:  

1. **Text Embedding:**  
   - The embedding model (**e.g., `sentence-transformers/all-MiniLM-L6-v2`**) converts each document (or chunk) into a high-dimensional numerical vector.  
   - This transformation captures the semantic meaning of the text.  

2. **Storing in the Vector Database:**  
   - The generated embeddings are stored in a **vector store** (e.g., **ChromaDB**) alongside metadata like document type.  
   - These embeddings act as **index entries**, allowing the system to quickly find semantically similar texts.  

In [ ]:
# Managing the Chroma Vector Store

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Indexing
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

The Chroma vector store is initialized. If an existing store is found, it is deleted to start fresh. The new store is populated with document chunks and their embeddings.

---

#### **Analyzing Embedding Dimensions**

Let's retrieves a sample embedding and prints its dimensionality.

In [ ]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

### **Let's create the complete RAG**

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In [ ]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)